In [ ]:
import pandas as pd
fea = pd.read_csv("../input/jane-street-market-prediction/train.csv")

In [ ]:
fea.head(10)

In [ ]:
fea.tail(15)

List of the columns

In [ ]:
fea.columns.tolist()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
COLS = fea.iloc[:,7:10].columns
for f in COLS:
    fig, axs = plt.subplots(1, 4, figsize=(15, 4))
    sns.distplot(fea[f], ax=axs[0])
    sns.distplot(fea.query('weight > 0')[f], ax=axs[1])
    try:
        sns.distplot(fea.query('weight > 0 and resp > 0')[f].dropna().apply(np.log1p), ax=axs[2])
        sns.distplot(fea.query('weight > 0 and resp < 0')[f].dropna().apply(np.log1p), ax=axs[2])
    except:
        pass
    fea.sample(5000).plot(kind='scatter', x=f, y='resp', ax=axs[3])
    fig.suptitle(f, fontsize=15, y=1.1)
    
    axs[0].set_title('feature distribution')
    axs[1].set_title('only weight > 0')
    axs[2].set_title('log transform')
    axs[3].set_title('feature vs. response')
    
    plt.tight_layout()
    plt.show()

**Utility Score**

Each row in the set represents a trading return for which we have to predict an action value, 1 to make the trade and 0 if there is no trade. Each of them have an associated weight and resp, which represents a return.
The variable pi is a indicator for each day i, showing how much return we got for that day.

   
  pi=∑j(weight∗resp∗1)
  
  t=∑pi∑p2i−−−−√∗250|i|
  
  u=min(max(t,0),6)∑pi
  
  where (|i|) is the number of unique dates in the test set. 


1. weight is given in both the training and testing data
2. resp is only given in the training data but not in the testing data
3. action is not given in both
 
The target variable is action.


Finding return for first row which has date = 0

In [ ]:
fea_first = fea[fea['date'] == 0]

In [ ]:
fea_first['mult'] = fea_first['weight']*fea_first['resp']*1
p_first = fea_first['mult'].sum()
p_first



Here for Date = 0 , we have return of score 12.

Let's find out what is the maximum return from we can obtain from day 499.

In [ ]:
fea_last = fea[fea['date'] == 499]

In [ ]:
fea_last['mult'] = fea_last['weight']*fea_last['resp']*(fea_last['resp'] > 0)
p_last = fea_last['mult'].sum()
p_last

The maximum score for date 499 is 114.45

Calculation of t:

In [ ]:
t = (np.sum(np.array([74, 2])/np.sqrt(np.sum(np.array([74,2])**2))))*np.sqrt(250/2)
t

**Checking missing data for Train**

In [ ]:
total = fea.isnull().sum().sort_values(ascending = False)
percent = (fea.isnull().sum()/fea.isnull().count()*100).sort_values(ascending = False)
missing__train_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing__train_data.head(10)


**Feature correlation**

In [ ]:
correlations = fea.corr(method='pearson')

fig, axs = plt.subplots(figsize=(16, 16))
sns.heatmap(correlations)
fig.savefig('correlation_map.png')

